# Momento 1

## BaseLine

### Potenciais Variações

### Dataset - feature engineering

### Método de aprendizado

### HiperParâmetros

#### Otimizadores

#### Técnica de convergência

#### Prunning

#### Normalização

#### Regularização

## Desempenho

# Momento 2

### Potenciais Variações

### Dataset - feature engineering

### Método de aprendizado

### HiperParâmetros

#### Otimizadores

#### Técnica de convergência

#### Prunning

#### Normalização

#### Regularização

## Desempenho